In [ ]:
import urllib.request as req
import os.path, random
import json
import requests

key = '9bd94b7e582fdba45a601ed68f788300'
url = 'http://kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json?key=9bd94b7e582fdba45a601ed68f788300&movieNm=쥬라기'

response = requests.get(url)

# 상태코드 200이면 정상 응답
if response.status_code == 200:
    data = response.json()  # JSON을 Python dict로 변환
    print(data)  # 전체 데이터 출력

    # 예: 영화 리스트 출력
    for movie in data["movieListResult"]["movieList"]:
        print(f"제목: {movie['movieNm']}, 장르: {movie['genreAlt']}")
else:
    print("요청 실패:", response.status_code)

# savename = "movieDB.json"

# # exists를 이용하여 경로에 해당하는 파일이 없으면
# # if문을 실행하여 url의 내용으로 파일 생성
# if not os.path.exists(savename):
#     req.urlretrieve(url, savename)

# # json 파일 분석
# items = json.load(open(savename, 'r', encoding='utf-8'))

# # s = oprn(savename, 'r', encoding*('utf-2).read()

{'movieListResult': {'totCnt': 39, 'source': '영화진흥위원회', 'movieList': [{'movieCd': '20252527', 'movieNm': '쥬라기 월드: 새로운 시작', 'movieNmEn': 'Jurassic World: Rebirth', 'prdtYear': '2025', 'openDt': '20250702', 'typeNm': '장편', 'prdtStatNm': '개봉', 'nationAlt': '미국', 'genreAlt': '어드벤처,액션,스릴러,공포(호러)', 'repNationNm': '미국', 'repGenreNm': '어드벤처', 'directors': [{'peopleNm': '가렛 에드워즈'}], 'companys': []}, {'movieCd': '2025A109', 'movieNm': '쥬라기 아일랜드', 'movieNmEn': 'Jurassic Park', 'prdtYear': '2024', 'openDt': '', 'typeNm': '장편', 'prdtStatNm': '기타', 'nationAlt': '중국', 'genreAlt': '액션', 'repNationNm': '중국', 'repGenreNm': '액션', 'directors': [], 'companys': []}, {'movieCd': '20242021', 'movieNm': '쥬라기캅스 극장판: 전설의 고대생물을 찾아라', 'movieNmEn': '', 'prdtYear': '2024', 'openDt': '20240815', 'typeNm': '장편', 'prdtStatNm': '개봉', 'nationAlt': '한국', 'genreAlt': '애니메이션', 'repNationNm': '한국', 'repGenreNm': '애니메이션', 'directors': [{'peopleNm': '이재민'}], 'companys': [{'companyCd': '20229242', 'companyNm': '(주)스튜디오버튼 '}]}, 

In [29]:
import requests
import json
from time import sleep
from datetime import datetime
import re

TMDB_API_KEY = '5349acaed8d7c3af060172341881019e'  # TMDb API 키
TMDB_BASE = "https://api.themoviedb.org/3"
MAX_MOVIES = 3000  # 최대 3000개로 제한
MOVIES_PER_PAGE = 20
MAX_PAGES = 1000  # 최대 페이지 수 (보통 500페이지면 10,000개 영화)

all_movies = []
today_str = datetime.now().strftime("%Y-%m-%d")

# 한글 포함 여부 체크 함수
def contains_hangul(text):
    if text is None:
        return False
    return bool(re.search(r'[가-힣]', text))

def get_director(movie_id):
    url = f"{TMDB_BASE}/movie/{movie_id}/credits"
    params = {"api_key": TMDB_API_KEY}
    res = requests.get(url, params=params)
    if res.status_code != 200:
        return ""
    data = res.json()
    directors = [c["name"] for c in data.get("crew", []) if c["job"] == "Director"]
    return directors[0] if directors else ""

for page in range(1, MAX_PAGES + 1):
    params = {
        "api_key": TMDB_API_KEY,
        "language": "ko",
        "sort_by": "primary_release_date.desc",
        "include_adult": False,
        "page": page,
        "primary_release_date.lte": today_str
    }

    response = requests.get(f"{TMDB_BASE}/discover/movie", params=params)
    if response.status_code != 200:
        print(f"페이지 {page} 요청 실패: {response.status_code}")
        break

    data = response.json()
    movies = data.get("results", [])
    if not movies:
        print(f"페이지 {page}에 데이터 없음, 중단")
        break

    for m in movies:
        movie_id = m.get("id")
        title_ko = m.get("title", "")

        # 한글 제목 아니면 건너뜀
        if not movie_id or not contains_hangul(title_ko):
            continue

        director = get_director(movie_id)
        genre_ids = m.get("genre_ids", [])

        movie_info = {
            "movie_id": movie_id,
            "title_ko": title_ko,
            "title_en": m.get("original_title", ""),
            "overview": m.get("overview", ""),
            "genres": genre_ids,
            "director": director,
            "vote_average": m.get("vote_average", 0),
            "release_date": m.get("release_date", ""),
            "poster_path": m.get("poster_path", "")
        }
        all_movies.append(movie_info)

        if len(all_movies) >= MAX_MOVIES:
            print(f"최대 영화 수 {MAX_MOVIES}개 도달, 중단")
            break

    print(f"페이지 {page} 완료, 누적 영화 수: {len(all_movies)}편")

    if len(all_movies) >= MAX_MOVIES:
        break

    sleep(0.3)

def get_genre_mapping():
    url = f"{TMDB_BASE}/genre/movie/list"
    params = {"api_key": TMDB_API_KEY, "language": "ko"}
    res = requests.get(url, params=params)
    if res.status_code != 200:
        return {}
    data = res.json()
    return {g['id']: g['name'] for g in data.get('genres', [])}

genre_map = get_genre_mapping()
for m in all_movies:
    genre_ids = m.get("genres", [])
    m["genres"] = [genre_map.get(gid, "") for gid in genre_ids]

with open("movies_DB.json", "w", encoding="utf-8") as f:
    json.dump(all_movies, f, ensure_ascii=False, indent=2)

print(f"✅ 저장 완료: 총 {len(all_movies)}편 → movies_DB.json")

페이지 1 완료, 누적 영화 수: 0편
페이지 2 완료, 누적 영화 수: 1편
페이지 3 완료, 누적 영화 수: 1편
페이지 4 완료, 누적 영화 수: 3편
페이지 5 완료, 누적 영화 수: 3편
페이지 6 완료, 누적 영화 수: 3편
페이지 7 완료, 누적 영화 수: 4편
페이지 8 완료, 누적 영화 수: 4편
페이지 9 완료, 누적 영화 수: 4편
페이지 10 완료, 누적 영화 수: 4편
페이지 11 완료, 누적 영화 수: 4편
페이지 12 완료, 누적 영화 수: 4편
페이지 13 완료, 누적 영화 수: 5편
페이지 14 완료, 누적 영화 수: 8편
페이지 15 완료, 누적 영화 수: 10편
페이지 16 완료, 누적 영화 수: 10편
페이지 17 완료, 누적 영화 수: 11편
페이지 18 완료, 누적 영화 수: 11편
페이지 19 완료, 누적 영화 수: 11편
페이지 20 완료, 누적 영화 수: 15편
페이지 21 완료, 누적 영화 수: 15편
페이지 22 완료, 누적 영화 수: 18편
페이지 23 완료, 누적 영화 수: 20편
페이지 24 완료, 누적 영화 수: 24편
페이지 25 완료, 누적 영화 수: 27편
페이지 26 완료, 누적 영화 수: 28편
페이지 27 완료, 누적 영화 수: 29편
페이지 28 완료, 누적 영화 수: 31편
페이지 29 완료, 누적 영화 수: 33편
페이지 30 완료, 누적 영화 수: 33편
페이지 31 완료, 누적 영화 수: 40편
페이지 32 완료, 누적 영화 수: 50편
페이지 33 완료, 누적 영화 수: 54편
페이지 34 완료, 누적 영화 수: 54편
페이지 35 완료, 누적 영화 수: 55편
페이지 36 완료, 누적 영화 수: 71편
페이지 37 완료, 누적 영화 수: 74편
페이지 38 완료, 누적 영화 수: 74편
페이지 39 완료, 누적 영화 수: 74편
페이지 40 완료, 누적 영화 수: 74편
페이지 41 완료, 누적 영화 수: 77편
페이지 42 완료, 누적 영화 수: 88편
페이지 43 완료, 누적 영

In [ ]:
# json파일에서 한글명이 있는 데이터로 다시 추출
import json
import re

# 한글 포함 여부 체크 함수
def contains_korean(text):
    if text is None:
        return False
    return bool(re.search('[가-힣]', text))

# 기존 JSON 파일 불러오기
with open("movies_DB.json", encoding="utf-8") as f:
    movies = json.load(f)

# 한글이 포함된 title_ko만 필터링
movies_with_korean_title = [movie for movie in movies if contains_korean(movie.get("title_ko", ""))]

print(f"한글이 포함된 제목의 영화 수: {len(movies_with_korean_title)}")

# 새 JSON 파일로 저장
with open("movies_with_korean_title.json", "w", encoding="utf-8") as f:
    json.dump(movies_with_korean_title, f, ensure_ascii=False, indent=2)

print("한글 제목이 포함된 영화 데이터만 저장 완료.")

한글이 포함된 제목의 영화 수: 540
한글 제목이 포함된 영화 데이터만 저장 완료.


In [26]:
import urllib.request as req
import os.path, random
import json

from konlpy.tag import Okt


PATH = "C:/kosmo/public/" # PATH 경로 
file_name = "movies_with_korean_title.json"
POST_URL = "https://image.tmdb.org/t/p/w500/"
with open(PATH+file_name, encoding="utf-8") as f:
    movies = json.load(f)
# print(movies)

# for movie in movies:
#     print("영화 제목 : ", movie['title_ko'])
#     print("줄거리 : ", movie['overview'])
#     print(f"장르 : {[g for g in movie['genres']]}")
#     print(f"포스터 : {POST_URL+str(movie['poster_path'])}")


def get_tokens(title, overview):
    okt = Okt()
    words = okt.pos(title, stem=True, norm=True)
    token = []
    for word, pos in words:
        if pos in ['Noun', 'Verb', 'Adjective']:
            token.append(word)
    words = okt.pos(overview, stem=True, norm=True)
    for word, pos in words:
        if pos in ['Noun', 'Verb', 'Adjective']:
            token.append(word)
    return token
keyword = []
# get_tokens("난장판이 된 사건사고: 열기구 소년 집에서 만든 비행접시가 날아오르며, 여섯 살짜리 꼬마를 싣고 갔다? 이 황당한 사고에 전 미국이 혼란에 빠지는데. 실제로는 어떤 일이 있었던 것일까. 이 다큐멘터리를 통해 알아보자.")
for movie in movies:
    keyword.append(get_tokens(movie['title_ko'], movie['overview']))
for k in keyword:
    print(k)

    

입력하신 '쥬라기'과(와) 가장 비슷한 영화 제목은 '해바라기'입니다.

'쥬라기' 추천 영화 리스트:
매드 마스크: 유사도 0.996
인류존속의 미스터리 그리고 두 연인: 유사도 0.996
스위트홈 감독판: 유사도 0.996
개구리는 죽는다: 유사도 0.996
신명: 유사도 0.996


In [32]:
import urllib.request as req
import os.path, random
import json
from konlpy.tag import Okt
from gensim.models import Word2Vec
import numpy as np

# --- 데이터 불러오기 및 경로 설정 ---
PATH = "C:/kosmo/test_data/IH/"
file_name = "movies_DB_partial.json"
POST_URL = "https://image.tmdb.org/t/p/w500/"
with open(PATH + file_name, encoding="utf-8") as f:
    movies = json.load(f)

# --- 형태소 토큰화 함수 (제목 + 개요) ---
okt = Okt()

def get_tokens(title, overview):
    title = title or ""
    overview = overview or ""
    token = []
    for word, pos in okt.pos(title, stem=True, norm=True):
        if pos in ['Noun', 'Verb', 'Adjective']:
            token.append(word)
    for word, pos in okt.pos(overview, stem=True, norm=True):
        if pos in ['Noun', 'Verb', 'Adjective']:
            token.append(word)
    return token

# --- 전체 영화 토큰 리스트 만들기 ---
keyword = []
for movie in movies:
    keyword.append(get_tokens(movie.get('title_ko'), movie.get('overview')))

# --- Word2Vec 모델 학습 ---
model = Word2Vec(keyword, vector_size=100, window=5, min_count=1, workers=4, sg=0)

# --- 영화 토큰 리스트를 임베딩 벡터로 변환 ---
def get_movie_vector(tokens, model):
    vectors = [model.wv[token] for token in tokens if token in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

# --- 전체 영화 벡터 리스트 생성 ---
movie_vectors = [get_movie_vector(tokens, model) for tokens in keyword]

# --- 코사인 유사도 함수 ---
def cosine_similarity(vec1, vec2):
    if np.linalg.norm(vec1) == 0 or np.linalg.norm(vec2) == 0:
        return 0.0
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# --- 제목만 토큰화 (형태소 기반 유사 제목 찾기용) ---
def tokenize_title(text):
    if not text:
        return []
    return [word for word, pos in okt.pos(text, stem=True, norm=True) if pos in ['Noun', 'Verb', 'Adjective']]

# --- Jaccard 유사도 ---
def jaccard_similarity(set1, set2):
    if not set1 or not set2:
        return 0.0
    return len(set1 & set2) / len(set1 | set2)

# --- 형태소 기반 유사 제목 찾기 ---
def find_similar_title_token_based(input_title, movies, threshold=0.1):
    input_tokens = set(tokenize_title(input_title))
    best_match = None
    best_score = 0.0
    for movie in movies:
        title = movie.get('title_ko')
        if not title:
            continue
        title_tokens = set(tokenize_title(title))
        score = jaccard_similarity(input_tokens, title_tokens)
        if score > best_score and score >= threshold:
            best_score = score
            best_match = title
    return best_match

# --- 최종 추천 함수 ---
def recommend_movie_token_based(input_title, movies, keyword, movie_vectors, model, top_n=5):
    matched_title = find_similar_title_token_based(input_title, movies)

    if matched_title is None:
        print(f"'{input_title}'과(와) 비슷한 영화 제목을 찾지 못했습니다. 전체 영화 평균으로 추천합니다.")
        input_vec = np.mean(movie_vectors, axis=0)
    else:
        print(f"입력하신 '{input_title}'과(와) 가장 비슷한 영화 제목은 '{matched_title}'입니다.")
        for m, tokens in zip(movies, keyword):
            if m.get('title_ko') == matched_title:
                input_vec = get_movie_vector(tokens, model)
                break

    similarity_scores = []
    for i, vec in enumerate(movie_vectors):
        sim = cosine_similarity(input_vec, vec)
        similarity_scores.append((i, sim))

    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    recommendations = []
    for idx, score in similarity_scores:
        if matched_title and movies[idx].get('title_ko') == matched_title:
            continue
        recommendations.append((movies[idx].get('title_ko'), score))
        if len(recommendations) >= top_n:
            break

    return recommendations

# --- 추천 테스트 ---
input_title = "쥬라기"
recs = recommend_movie_token_based(input_title, movies, keyword, movie_vectors, model)

print(f"\n'{input_title}' 추천 영화 리스트:")
for title, score in recs:
    print(f"{title}: 유사도 {score:.3f}")

KeyboardInterrupt: 

In [30]:
import requests
import json
from time import sleep
from datetime import datetime, timedelta

TMDB_API_KEY = '5349acaed8d7c3af060172341881019e'
TMDB_BASE = "https://api.themoviedb.org/3"
POSTER_URL = "https://image.tmdb.org/t/p/w500"
MAX_PAGES = 500  # TMDB 최대 페이지 제한

def get_director(movie_id):
    try:
        url = f"{TMDB_BASE}/movie/{movie_id}/credits"
        params = {"api_key": TMDB_API_KEY}
        res = requests.get(url, params=params)
        if res.status_code != 200:
            return ""
        data = res.json()
        directors = [c["name"] for c in data.get("crew", []) if c["job"] == "Director"]
        return directors[0] if directors else ""
    except:
        return ""

def get_genre_mapping():
    url = f"{TMDB_BASE}/genre/movie/list"
    params = {"api_key": TMDB_API_KEY, "language": "ko"}
    res = requests.get(url, params=params)
    if res.status_code != 200:
        return {}
    data = res.json()
    return {g['id']: g['name'] for g in data.get('genres', [])}

def fetch_movies_by_month(year, month, end_date_override=None):
    all_movies = []
    genre_map = get_genre_mapping()
    start_date = datetime(year, month, 1)
    if end_date_override:
        end_date = end_date_override
    else:
        if month == 12:
            end_date = datetime(year + 1, 1, 1) - timedelta(days=1)
        else:
            end_date = datetime(year, month + 1, 1) - timedelta(days=1)

    page = 1
    while page <= MAX_PAGES:
        try:
            params = {
                "api_key": TMDB_API_KEY,
                "language": "ko",
                "sort_by": "primary_release_date.asc",
                "include_adult": False,
                "page": page,
                "primary_release_date.gte": start_date.strftime("%Y-%m-%d"),
                "primary_release_date.lte": end_date.strftime("%Y-%m-%d"),
            }
            res = requests.get(f"{TMDB_BASE}/discover/movie", params=params)
            if res.status_code != 200:
                print(f"❌ {year}-{month:02d} 페이지 {page} 요청 실패: {res.status_code}")
                break

            data = res.json()
            movies = data.get("results", [])
            if not movies:
                break

            for m in movies:
                title_ko = m.get("title", "")
                if not any('\uac00' <= ch <= '\ud7a3' for ch in title_ko):  # 한글 포함 체크
                    continue

                movie_id = m.get("id")
                director = get_director(movie_id)
                genre_ids = m.get("genre_ids", [])
                genres = [genre_map.get(gid, "") for gid in genre_ids]

                movie_info = {
                    "movie_id": movie_id,
                    "title_ko": title_ko,
                    "title_en": m.get("original_title", ""),
                    "overview": m.get("overview", ""),
                    "genres": genres,
                    "director": director,
                    "vote_average": m.get("vote_average", 0),
                    "release_date": m.get("release_date", ""),
                    "poster_path": POSTER_URL + m.get("poster_path") if m.get("poster_path") else "",
                }
                all_movies.append(movie_info)

            print(f"✅ {year}-{month:02d} 페이지 {page} 완료 (누적: {len(all_movies)}편)")
            page += 1
            sleep(0.5)
        except Exception as e:
            print(f"🚨 예외 발생: {e}")
            break

    return all_movies

def collect_movies(start_year, start_month, end_year, end_month):
    collected = {}
    y, m = start_year, start_month
    today = datetime.now()

    try:
        while (y < end_year) or (y == end_year and m <= end_month):
            print(f"\n📅 {y}-{m:02d} 수집 시작")
            if y == end_year and m == end_month:
                end_date_override = datetime(today.year, today.month, today.day)
            else:
                end_date_override = None

            movies = fetch_movies_by_month(y, m, end_date_override)
            for movie in movies:
                collected[movie["movie_id"]] = movie

            if m == 12:
                y += 1
                m = 1
            else:
                m += 1
    except Exception as e:
        print(f"⚠️ 중간 오류 발생: {e}")
    finally:
        filename = "movies_DB_partial.json"
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(list(collected.values()), f, ensure_ascii=False, indent=2)
        print(f"\n📝 수집 데이터 {len(collected)}편 저장 완료 → {filename}")

    return list(collected.values())

# 🔽 실행: 2000년 1월 ~ 2025년 7월 15일까지 수집
movies_data = collect_movies(2000, 1, 2025, 7)


📅 2000-01 수집 시작
✅ 2000-01 페이지 1 완료 (누적: 1편)
✅ 2000-01 페이지 2 완료 (누적: 1편)
✅ 2000-01 페이지 3 완료 (누적: 1편)
✅ 2000-01 페이지 4 완료 (누적: 2편)
✅ 2000-01 페이지 5 완료 (누적: 2편)
✅ 2000-01 페이지 6 완료 (누적: 2편)
✅ 2000-01 페이지 7 완료 (누적: 2편)
✅ 2000-01 페이지 8 완료 (누적: 2편)
✅ 2000-01 페이지 9 완료 (누적: 2편)
✅ 2000-01 페이지 10 완료 (누적: 2편)
✅ 2000-01 페이지 11 완료 (누적: 2편)
✅ 2000-01 페이지 12 완료 (누적: 2편)
✅ 2000-01 페이지 13 완료 (누적: 3편)
✅ 2000-01 페이지 14 완료 (누적: 3편)
✅ 2000-01 페이지 15 완료 (누적: 3편)
✅ 2000-01 페이지 16 완료 (누적: 3편)
✅ 2000-01 페이지 17 완료 (누적: 3편)
✅ 2000-01 페이지 18 완료 (누적: 3편)
✅ 2000-01 페이지 19 완료 (누적: 3편)
✅ 2000-01 페이지 20 완료 (누적: 3편)
✅ 2000-01 페이지 21 완료 (누적: 3편)
✅ 2000-01 페이지 22 완료 (누적: 3편)
✅ 2000-01 페이지 23 완료 (누적: 3편)
✅ 2000-01 페이지 24 완료 (누적: 3편)
✅ 2000-01 페이지 25 완료 (누적: 3편)
✅ 2000-01 페이지 26 완료 (누적: 3편)
✅ 2000-01 페이지 27 완료 (누적: 3편)
✅ 2000-01 페이지 28 완료 (누적: 3편)
✅ 2000-01 페이지 29 완료 (누적: 3편)
✅ 2000-01 페이지 30 완료 (누적: 3편)
✅ 2000-01 페이지 31 완료 (누적: 3편)
✅ 2000-01 페이지 32 완료 (누적: 4편)
✅ 2000-01 페이지 33 완료 (누적: 4편)
✅ 2000-01 페이지 34 완료 (누적: 5편)
✅ 2000

In [ ]:
import pickle

# --- 토큰 생성 시 장르 포함 ---
def get_tokens_with_genre(title, overview, genres):
    title = title or ""
    overview = overview or ""
    tokens = []
    for word, pos in okt.pos(title, stem=True, norm=True):
        if pos in ['Noun', 'Verb', 'Adjective']:
            tokens.append(word)
    for word, pos in okt.pos(overview, stem=True, norm=True):
        if pos in ['Noun', 'Verb', 'Adjective']:
            tokens.append(word)
    # 장르도 토큰에 추가
    tokens.extend([g for g in genres if g])
    return tokens

# --- 전체 영화 토큰 리스트 만들기 ---
keyword = [get_tokens_with_genre(m.get('title_ko'), m.get('overview'), m.get('genres', [])) for m in movies]

# --- Word2Vec 모델 학습 ---
model = Word2Vec(keyword, vector_size=100, window=5, min_count=1, workers=4, sg=0)

# --- 영화 벡터 리스트 생성 ---
movie_vectors = [get_movie_vector(tokens, model) for tokens in keyword]

# --- pickle로 저장 ---
with open('movie_w2v_model.pkl', 'wb') as f:
    pickle.dump(model, f)
with open('movie_vectors.pkl', 'wb') as f:
    pickle.dump(movie_vectors, f)
with open('movie_keywords.pkl', 'wb') as f:
    pickle.dump(keyword, f)
with open('movie_list.pkl', 'wb') as f:
    pickle.dump(movies, f)

# --- pickle로 로드 및 추천 함수 ---
def load_and_recommend(input_title, top_n=5):
    with open('movie_w2v_model.pkl', 'rb') as f:
        model = pickle.load(f)
    with open('movie_vectors.pkl', 'rb') as f:
        movie_vectors = pickle.load(f)
    with open('movie_keywords.pkl', 'rb') as f:
        keyword = pickle.load(f)
    with open('movie_list.pkl', 'rb') as f:
        movies = pickle.load(f)
    return recommend_movie_token_based(input_title, movies, keyword, movie_vectors, model, top_n)

# --- 사용 예시 ---
recs = load_and_recommend("쥬라기")
for title, score in recs:
    print(f"{title}: 유사도 {score:.3f}")


In [34]:
# 추천 시스템 구현
import numpy as np
from konlpy.tag import Okt
import joblib

PATH = "C:/kosmo/test_data/IH/"

# --- 저장한 데이터 및 모델 로드 ---
model = joblib.load(PATH + "word2vec_movie.model")
movie_vectors = joblib.load(PATH + "movie_vectors.pkl")
movies = joblib.load(PATH + "movies.pkl")
keyword = joblib.load(PATH + "keyword.pkl")

okt = Okt()

# --- 유사도 계산 함수 ---
def cosine_similarity(vec1, vec2):
    if np.linalg.norm(vec1) == 0 or np.linalg.norm(vec2) == 0:
        return 0.0
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# --- 형태소 기반 제목 토큰화 ---
def tokenize_title(text):
    if not text:
        return []
    return [word for word, pos in okt.pos(text, stem=True, norm=True) if pos in ['Noun', 'Verb', 'Adjective']]

# --- Jaccard 유사도 ---
def jaccard_similarity(set1, set2):
    if not set1 or not set2:
        return 0.0
    return len(set1 & set2) / len(set1 | set2)

# --- 가장 유사한 영화 객체 찾기 ---
def find_similar_movie(input_title, movies, threshold=0.1):
    input_tokens = set(tokenize_title(input_title))
    best_match = None
    best_score = 0.0
    for movie in movies:
        title = movie.get('title_ko')
        if not title:
            continue
        title_tokens = set(tokenize_title(title))
        score = jaccard_similarity(input_tokens, title_tokens)
        if score > best_score and score >= threshold:
            best_score = score
            best_match = movie
    return best_match

# --- 영화 벡터 구하기 ---
def get_movie_vector(tokens, model):
    vectors = [model.wv[token] for token in tokens if token in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

# --- 추천 함수 ---
def recommend_movie(input_title, movies, keyword, movie_vectors, model, top_n=5):
    matched_movie = find_similar_movie(input_title, movies)

    if matched_movie is None:
        print(f"'{input_title}'과(와) 비슷한 영화를 찾지 못했습니다. 전체 영화 평균으로 추천합니다.")
        input_vec = np.mean(movie_vectors, axis=0)
    else:
        print(f"입력하신 '{input_title}'과(와) 가장 비슷한 영화는 '{matched_movie.get('title_ko')}'입니다.")
        idx = movies.index(matched_movie)
        input_vec = movie_vectors[idx]

    similarity_scores = []
    for i, vec in enumerate(movie_vectors):
        if matched_movie and i == movies.index(matched_movie):
            continue  # 자기 자신 제외
        sim = cosine_similarity(input_vec, vec)
        similarity_scores.append((i, sim))

    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    recommendations = []
    for idx, score in similarity_scores[:top_n]:
        recommendations.append((movies[idx].get('title_ko'), score))

    return recommendations

# --- 추천 테스트 ---
input_title = "쥬라기"
recs = recommend_movie(input_title, movies, keyword, movie_vectors, model)

print(f"\n'{input_title}' 추천 영화 리스트:")
for title, score in recs:
    print(f"{title}: 유사도 {score:.3f}")

입력하신 '쥬라기'과(와) 가장 비슷한 영화는 '쥬라기 공원 3'입니다.

'쥬라기' 추천 영화 리스트:
아틀란티스 2: 마일로의 귀환: 유사도 0.948
킹: 잃어버린 세계: 유사도 0.947
혹성탈출: 유사도 0.943
에이리언 터미네이터: 유사도 0.941
아이스 에이지: 유사도 0.936


In [35]:
import numpy as np
from konlpy.tag import Okt
import joblib

PATH = "C:/kosmo/test_data/IH/"

# --- 저장한 데이터 및 모델 로드 ---
model = joblib.load(PATH + "word2vec_movie.model")
movie_vectors = joblib.load(PATH + "movie_vectors.pkl")
movies = joblib.load(PATH + "movies.pkl")
keyword = joblib.load(PATH + "keyword.pkl")

okt = Okt()

def tokenize_title(text):
    if not text:
        return []
    return [word for word, pos in okt.pos(text, stem=True, norm=True) if pos in ['Noun', 'Verb', 'Adjective']]

def jaccard_similarity(set1, set2):
    if not set1 or not set2:
        return 0.0
    return len(set1 & set2) / len(set1 | set2)

def find_similar_movie(input_title, movies, threshold=0.1):
    input_tokens = set(tokenize_title(input_title))
    best_match = None
    best_score = 0.0
    for movie in movies:
        title = movie.get('title_ko')
        if not title:
            continue
        title_tokens = set(tokenize_title(title))
        score = jaccard_similarity(input_tokens, title_tokens)
        if score > best_score and score >= threshold:
            best_score = score
            best_match = movie
    return best_match

def recommend_movie(input_title, movies, keyword, movie_vectors, model, top_n=5):
    matched_movie = find_similar_movie(input_title, movies)

    if matched_movie is None:
        print(f"'{input_title}'과(와) 비슷한 영화를 찾지 못했습니다. 전체 영화 평균으로 추천합니다.")
        input_vec = np.mean(movie_vectors, axis=0)
    else:
        print(f"입력하신 '{input_title}'과(와) 가장 비슷한 영화는 '{matched_movie.get('title_ko')}'입니다.")
        idx = movies.index(matched_movie)
        input_vec = movie_vectors[idx]

    vectors = np.array(movie_vectors)
    input_norm = np.linalg.norm(input_vec)
    vectors_norm = np.linalg.norm(vectors, axis=1)
    dot_products = vectors @ input_vec

    with np.errstate(divide='ignore', invalid='ignore'):
        cosine_similarities = dot_products / (vectors_norm * input_norm)
        cosine_similarities = np.nan_to_num(cosine_similarities)

    if matched_movie:
        cosine_similarities[movies.index(matched_movie)] = -1

    top_indices = np.argsort(cosine_similarities)[::-1][:top_n]
    recommendations = []
    for i in top_indices:
        m = movies[i]
        rec = {
            "title_ko": m.get('title_ko', ''),
            "release_date": m.get('release_date', ''),
            "vote_average": m.get('vote_average', 0),
            "poster_path": m.get('poster_path', ''),
            "similarity": cosine_similarities[i]
        }
        recommendations.append(rec)

    return recommendations

# --- 추천 테스트 ---
input_title = "쥬라기"
recs = recommend_movie(input_title, movies, keyword, movie_vectors, model)

print(f"\n'{input_title}' 추천 영화 리스트:")
for rec in recs:
    print(f"{rec['title_ko']} (개봉일: {rec['release_date']}, 평점: {rec['vote_average']:.1f})")
    print(f"포스터: {rec['poster_path']}")
    print(f"유사도: {rec['similarity']:.3f}\n")

입력하신 '쥬라기'과(와) 가장 비슷한 영화는 '쥬라기 공원 3'입니다.

'쥬라기' 추천 영화 리스트:
아틀란티스 2: 마일로의 귀환 (개봉일: 2003-02-25, 평점: 6.3)
포스터: https://image.tmdb.org/t/p/w500/z9VHUEuUtWcUZeKMGfd59886PdM.jpg
유사도: 0.948

킹: 잃어버린 세계 (개봉일: 2005-12-13, 평점: 4.3)
포스터: https://image.tmdb.org/t/p/w500/2wWIbl85HcxvUjPjLU6Qi4dStWn.jpg
유사도: 0.947

혹성탈출 (개봉일: 2001-07-25, 평점: 5.8)
포스터: https://image.tmdb.org/t/p/w500/gUYQbokoI2Pwng3JM9igtEmCcNo.jpg
유사도: 0.943

에이리언 터미네이터 (개봉일: 2011-02-12, 평점: 3.5)
포스터: https://image.tmdb.org/t/p/w500/aLdYYQ6HdlmNxJD91WhUXqJgg54.jpg
유사도: 0.941

아이스 에이지 (개봉일: 2002-03-14, 평점: 7.4)
포스터: https://image.tmdb.org/t/p/w500/biwqSbuyxjrZMvVcb8Lsv3GyQbw.jpg
유사도: 0.936



In [45]:
# 7월 16일 13:16 학습
import json
from konlpy.tag import Okt
from gensim.models import Word2Vec
import numpy as np
import joblib

PATH = "C:/kosmo/test_data/IH/"
file_name = "movies_DB_partial.json"

with open(PATH + file_name, encoding="utf-8") as f:
    movies = json.load(f)

okt = Okt()

def get_tokens(title, overview, genres):
    title = title or ""
    overview = overview or ""
    genres = genres or []
    tokens = []
    for word, pos in okt.pos(title, stem=True, norm=True):
        if pos in ['Noun', 'Verb', 'Adjective']:
            tokens.append(word)
    for word, pos in okt.pos(overview, stem=True, norm=True):
        if pos in ['Noun', 'Verb', 'Adjective']:
            tokens.append(word)
    tokens.extend(genres)
    return tokens

keyword = []
for movie in movies:
    tokens = get_tokens(movie.get('title_ko'), movie.get('overview'), movie.get('genres'))
    keyword.append(tokens)

model = Word2Vec(keyword, vector_size=100, window=5, min_count=1, workers=4, sg=0)  # min_count=1로 변경

title_tokens = [get_tokens(movie.get('title_ko'), "", []) for movie in movies]
joblib.dump(title_tokens, PATH + "title_tokens.pkl")

def get_movie_vector(tokens, model):
    vectors = [model.wv[token] for token in tokens if token in model.wv]
    return np.mean(np.array(vectors), axis=0) if vectors else np.zeros(model.vector_size)

movie_vectors = [get_movie_vector(tokens, model) for tokens in keyword]

joblib.dump(model, PATH + "word2vec_movie.model")
joblib.dump(movie_vectors, PATH + "movie_vectors.pkl")
joblib.dump(movies, PATH + "movies.pkl")
joblib.dump(keyword, PATH + "keyword.pkl")

print("✅ 학습 및 저장 완료")

✅ 학습 및 저장 완료


In [1]:
import platform
platform.architecture()

('64bit', 'WindowsPE')

In [1]:
%pip install jpype1

import jpype

print("Default JVM Path:", jpype.getDefaultJVMPath())

Defaulting to user installation because normal site-packages is not writeable
  Using cached jpype1-1.6.0-cp312-cp312-win_amd64.whl.metadata (5.1 kB)
Using cached jpype1-1.6.0-cp312-cp312-win_amd64.whl (355 kB)
Note: you may need to restart the kernel to use updated packages.
Default JVM Path: C:\Program Files\Java\jdk11\bin\server\jvm.dll


In [4]:
import joblib
import numpy as np
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

# 경로
PATH = "C:/kosmo/model/"

# 모델 및 데이터 불러오기
model = joblib.load(PATH + "word2vec_movie.model")
movie_vectors = joblib.load(PATH + "movie_vectors.pkl")
movies = joblib.load(PATH + "movies.pkl")
title_tokens = joblib.load(PATH + "title_tokens.pkl")

# 입력 영화 제목
input_title = "쥬라기 월드"  # 예시 제목

# 해당 제목에 맞는 토큰 찾기
input_index = None
for idx, movie in enumerate(movies):
    if movie.get('title_ko') == input_title:
        input_index = idx
        break

if input_index is None:
    print("❌ 해당 제목의 영화가 데이터에 없습니다.")
else:
    input_tokens = title_tokens[input_index]
    
    # 입력 영화의 벡터 구하기
    vectors = [model.wv[token] for token in input_tokens if token in model.wv]
    input_vector = np.mean(np.array(vectors), axis=0) if vectors else np.zeros(model.vector_size)
    
    # 유사도 계산
    similarities = cosine_similarity([input_vector], movie_vectors)[0]
    
    # 유사도 상위 5개 (자기 자신 제외)
    similar_idx = similarities.argsort()[::-1]
    top_indices = [i for i in similar_idx if i != input_index][:5]
    
    print(f"🎬 『{input_title}』와 유사한 영화 추천:")
    for idx in top_indices:
        print(f"- {movies[idx]['title_ko']} (유사도: {similarities[idx]:.4f})")

🎬 『쥬라기 월드』와 유사한 영화 추천:
- 패러렐 월드 (유사도: 0.9893)
- 스톰 시티 (유사도: 0.8998)
- 킹 오브 프리즘 - 드라마틱 프리즘.1 (유사도: 0.8850)
- 원더풀 월드 오브 디즈니: 인어공주 라이브! (유사도: 0.8788)
- 핑크무비 (유사도: 0.8700)


In [24]:
# main.py
import sys
import json
import numpy as np
import joblib
from konlpy.tag import Okt

# Jupyter 환경에서는 별도의 UTF-8 인코딩 설정이 필요하지 않습니다.
# sys.stdout = open(sys.stdout.fileno(), mode='w', encoding='utf-8')

PATH = "C:/kosmo/model/"

# --- 저장한 데이터 및 모델 로드 ---
model = joblib.load(PATH + "word2vec_movie.model")
movie_vectors = np.array(joblib.load(PATH + "movie_vectors.pkl"))
movies = joblib.load(PATH + "movies.pkl")
title_tokens = joblib.load(PATH + "title_tokens.pkl")  # <- 새로 저장된 제목 토큰

okt = Okt()

def tokenize_title(text):
    if not text:
        return []
    return [word for word, pos in okt.pos(text, stem=True, norm=True) if pos in ['Noun', 'Verb', 'Adjective']]

def jaccard_similarity(set1, set2):
    if not set1 or not set2:
        return 0.0
    return len(set1 & set2) / len(set1 | set2)

def find_similar_movie(input_title, movies, title_tokens, threshold=0.1):
    input_tokens = set(tokenize_title(input_title))
    best_match_idx = -1
    best_score = 0.0
    for idx, tokens in enumerate(title_tokens):
        score = jaccard_similarity(input_tokens, set(tokens))
        if score > best_score and score >= threshold:
            best_score = score
            best_match_idx = idx
    return best_match_idx

def recommend_movie(input_title, movies, movie_vectors, title_tokens, top_n=5):
    matched_idx = find_similar_movie(input_title, movies, title_tokens)

    if matched_idx == -1:
        print(f"'{input_title}'과(와) 비슷한 영화를 찾지 못했습니다. 전체 평균으로 추천합니다.")
        input_vec = np.mean(movie_vectors, axis=0)
    else:
        print(f"입력하신 '{input_title}'과(와) 가장 비슷한 영화는 '{movies[matched_idx]['title_ko']}'입니다.")
        input_vec = movie_vectors[matched_idx]

    input_norm = np.linalg.norm(input_vec)
    vectors_norm = np.linalg.norm(movie_vectors, axis=1)
    dot_products = movie_vectors @ input_vec

    with np.errstate(divide='ignore', invalid='ignore'):
        cosine_similarities = dot_products / (vectors_norm * input_norm)
        cosine_similarities = np.nan_to_num(cosine_similarities)

    if matched_idx >= 0:
        cosine_similarities[matched_idx] = -1

    top_indices = np.argpartition(-cosine_similarities, top_n)[:top_n]
    top_indices = top_indices[np.argsort(-cosine_similarities[top_indices])]

    recommendations = []
    for i in top_indices:
        m = movies[i]
        recommendations.append({
            "title_ko": m.get('title_ko', ''),
            "release_date": m.get('release_date', ''),
            "vote_average": m.get('vote_average', 0),
            "poster_path": m.get('poster_path', ''),
            "similarity": cosine_similarities[i]
        })

    return recommendations

if __name__ == "__main__":
    if len(sys.argv) < 2:
        print(json.dumps({"error": "텍스트 인자가 필요합니다."}))
        sys.exit(1)
    
    # search_text = sys.argv[1]  # Node.js에서 전달된 텍스트 인자
    print(sys.argv)

    input_title = "스타워즈"

    input_title = input_title.strip()  # 앞뒤 공백 제거
    # input_title = input_title.replace(" ", "")  # 띄어쓰기 제거하고 싶으면 주석 해제

    recs = recommend_movie(input_title, movies, movie_vectors, title_tokens, top_n=5)

    print(f"\n'{input_title}' 추천 영화 리스트:")
    for rec in recs:
        print(f"{rec['title_ko']} (개봉일: {rec['release_date']}, 평점: {rec['vote_average']:.1f})")
        print(f"포스터: {rec['poster_path']}")
        print(f"유사도: {rec['similarity']:.3f}\n")

['C:\\Users\\user\\AppData\\Roaming\\Python\\Python312\\site-packages\\ipykernel_launcher.py', '--f=c:\\Users\\user\\AppData\\Roaming\\jupyter\\runtime\\kernel-v3331b7541b998978fb2cea950a4c331ac52443869.json']
입력하신 '스타워즈'과(와) 가장 비슷한 영화는 '스타워즈 바이옴'입니다.

'스타워즈' 추천 영화 리스트:
제로베이스원 더 퍼스트 투어 [타임리스 월드] 인 시네마 (개봉일: 2025-03-12, 평점: 0.0)
포스터: https://image.tmdb.org/t/p/w500/uaL3Wug4YsZnMH0UAaIwwt1473C.jpg
유사도: 0.895

쿠렌치스 인 아테네 (개봉일: 2022-02-24, 평점: 10.0)
포스터: https://image.tmdb.org/t/p/w500/1bvN3PlPfzsWpXdxMJtV521FaUb.jpg
유사도: 0.890

팝 역사상 가장 위대한 밤 (개봉일: 2024-01-19, 평점: 7.9)
포스터: https://image.tmdb.org/t/p/w500/6Fe2vWvIImwoxsV5iq9pCvMTmcF.jpg
유사도: 0.887

500일의 다이앤 (개봉일: 2024-03-01, 평점: 6.0)
포스터: https://image.tmdb.org/t/p/w500/zcOTtSjpdSaMldbBTiGCVUIcsk4.jpg
유사도: 0.885

정동원 성탄총동원 더 무비 (개봉일: 2024-12-18, 평점: 0.0)
포스터: https://image.tmdb.org/t/p/w500/tSvNY471ORGm5g1gWUQDaKYrS1X.jpg
유사도: 0.882



In [7]:
%pip install soynlp

Defaulting to user installation because normal site-packages is not writeable
  Using cached soynlp-0.0.493-py3-none-any.whl.metadata (24 kB)
Using cached soynlp-0.0.493-py3-none-any.whl (416 kB)
Note: you may need to restart the kernel to use updated packages.


In [27]:
import sys
import json
import numpy as np
import joblib
from soynlp.tokenizer import LTokenizer

# 모델 경로
PATH = "C:/kosmo/model/"

# --- 모델 및 데이터 불러오기 ---
model = joblib.load(PATH + "word2vec_movie.model")
movie_vectors = np.array(joblib.load(PATH + "movie_vectors.pkl"))  # 줄거리+장르 벡터
movies = joblib.load(PATH + "movies.pkl")
title_tokens = joblib.load(PATH + "title_tokens.pkl")  # 제목 토큰 (검색용)

# soynlp 토크나이저 준비 (제목 검색용)
tokenizer = LTokenizer()

def tokenize_title(text):
    if not text:
        return []
    return tokenizer.tokenize(text)

def jaccard_similarity(set1, set2):
    if not set1 or not set2:
        return 0.0
    return len(set1 & set2) / len(set1 | set2)

def find_similar_movie(input_title, movies, title_tokens, threshold=0.1):
    # 제목 토큰 기준으로 유사한 영화 인덱스 찾기 (검색용)
    input_tokens = set(tokenize_title(input_title))
    best_match_idx = -1
    best_score = 0.0
    for idx, tokens in enumerate(title_tokens):
        score = jaccard_similarity(input_tokens, set(tokens))
        if score > best_score and score >= threshold:
            best_score = score
            best_match_idx = idx
    return best_match_idx

def recommend_movie(input_title, movies, movie_vectors, title_tokens, top_n=5):
    # 1) 제목 기준으로 비슷한 영화 인덱스 찾기 (검색)
    matched_idx = find_similar_movie(input_title, movies, title_tokens)

    # 2) matched_idx가 없으면 전체 벡터 평균으로 대체
    if matched_idx == -1:
        print(f"'{input_title}'과(와) 비슷한 영화를 찾지 못했습니다. 전체 평균 벡터로 추천합니다.")
        input_vec = np.mean(movie_vectors, axis=0)  # 줄거리+장르 벡터 평균
    else:
        print(f"입력하신 '{input_title}'과(와) 가장 비슷한 영화는 '{movies[matched_idx]['title_ko']}'입니다.")
        input_vec = movie_vectors[matched_idx]  # 줄거리+장르 벡터

    # 3) 코사인 유사도 계산 (줄거리+장르 벡터 기준)
    input_norm = np.linalg.norm(input_vec)
    vectors_norm = np.linalg.norm(movie_vectors, axis=1)
    dot_products = movie_vectors @ input_vec

    with np.errstate(divide='ignore', invalid='ignore'):
        cosine_similarities = dot_products / (vectors_norm * input_norm)
        cosine_similarities = np.nan_to_num(cosine_similarities)

    # 자기 자신 제외
    if matched_idx >= 0:
        cosine_similarities[matched_idx] = -1

    # 상위 top_n 개 인덱스 추출
    top_indices = np.argpartition(-cosine_similarities, top_n)[:top_n]
    top_indices = top_indices[np.argsort(-cosine_similarities[top_indices])]

    recommendations = []
    for i in top_indices:
        m = movies[i]
        recommendations.append({
            "title_ko": m.get('title_ko', ''),
            "release_date": m.get('release_date', ''),
            "vote_average": m.get('vote_average', 0),
            "poster_path": m.get('poster_path', ''),
            "similarity": cosine_similarities[i]
        })

    return recommendations

if __name__ == "__main__":
    input_title = "인셉션"

    recs = recommend_movie(input_title, movies, movie_vectors, title_tokens, top_n=5)

    print(f"\n'{input_title}' 추천 영화 리스트:")
    for rec in recs:
        print(f"{rec['title_ko']} (개봉일: {rec['release_date']}, 평점: {rec['vote_average']:.1f})")
        print(f"포스터: {rec['poster_path']}")
        print(f"유사도: {rec['similarity']:.3f}\n")

입력하신 '인셉션'과(와) 가장 비슷한 영화는 '인셉션'입니다.

'인셉션' 추천 영화 리스트:
스워드피쉬 (개봉일: 2001-06-08, 평점: 6.3)
포스터: https://image.tmdb.org/t/p/w500/63YY0GuORn1G73SAIADVDATdF5A.jpg
유사도: 0.951

드림메이커 (개봉일: 2021-11-17, 평점: 5.0)
포스터: https://image.tmdb.org/t/p/w500/4mgKEiVza796dzgzJjit6uTJrwW.jpg
유사도: 0.949

에반게리온: Q (개봉일: 2012-11-17, 평점: 7.2)
포스터: https://image.tmdb.org/t/p/w500/zERMhjBDiYoaHGDRsKP6IxzuV5F.jpg
유사도: 0.946

벌새 프로젝트 (개봉일: 2019-03-15, 평점: 6.2)
포스터: https://image.tmdb.org/t/p/w500/cx7CuTtcJvjZAP6ZFWoxylCXKFP.jpg
유사도: 0.944

서복 (개봉일: 2021-04-12, 평점: 7.2)
포스터: https://image.tmdb.org/t/p/w500/6R3eKsdVS7ZWrY28QUt3DCG3N72.jpg
유사도: 0.944

